In [1]:
import itertools
import re
import json
import random
import time
import copy

In [2]:
class CNF:
    def __init__(self, dimacs):
        dimacs_tokens = re.split('\s+', dimacs)
        self.num_of_vars = int(dimacs_tokens[2])
        self.num_of_clauses = int(dimacs_tokens[3])
        clauses_dimacs = [int(x) for x in dimacs_tokens[4:]]
        self.clauses = [list(clause) for is_zero, clause 
                    in itertools.groupby(clauses_dimacs, lambda x: x == 0) 
                    if not is_zero]
        
    def evaluate(self, valuation):
        for clause in self.clauses:
            clause_sat = False
            for literal in clause:
                val_idx = abs(literal) - 1
                if literal > 0 and valuation[val_idx] or literal < 0 and not valuation[val_idx]:
                    clause_sat = True
                    break
            if not clause_sat:
                return False
        return True

In [3]:
class Problem:
    def __init__(self, file_name):
        try:
            with open(file_name, 'r') as f:
                dimacs_list = json.load(f)
                self.cnf_list = [CNF(dimacs) for dimacs in dimacs_list]
                self.num_of_vars = self.cnf_list[0].num_of_vars
        except IOError:
            print(f'Error opening file {file_name}')
            exit(1)

In [4]:
class Solution:
    def __init__(self, problem, code=None):
        self.problem = problem
        if code is None:
            self.code = random.choices([True, False], k=self.problem.num_of_vars)
        else:
            self.code = code
        self.fitness = self.calc_fitness()
        
    def __lt__(self, other):
        return self.fitness < other.fitness
    
    def __str__(self):
        return f'Code: {self.code}, Number of satisfied: {self.get_num_sat()}'
    
    def calc_fitness(self):
        num_of_sat = 0
        for formula in self.problem.cnf_list:
            if formula.evaluate(self.code):
                num_of_sat += 1
        return 1 / (num_of_sat + 1)
    
    def get_num_sat(self):
        return round(1 / self.fitness - 1)
    
    def get_inverted(self, k=1):
        new_code = self.code
        for idx in random.sample(range(len(new_code)), k=k):
            new_code[idx] = not new_code[idx]
        return Solution(self.problem, new_code)

In [5]:
class BFSolver:
    @staticmethod
    def solve(problem):
        return max([Solution(problem, valuation) for valuation in 
                    itertools.product([True, False], repeat=problem.num_of_vars)]), 2**problem.num_of_vars

In [27]:
class GeneticSolver:
    def __init__(self, num_of_generations=50, max_without_improvement=20, population_size=10, mutation_probability=0.05, elitism=0.2):
        self.num_of_generations = num_of_generations
        self.max_without_improvement = max_without_improvement
        self.population_size = population_size
        self.mutation_probability = mutation_probability
        self.elitism = elitism

    def _generate_population(self, problem):
        return [Solution(problem) for _ in range(self.population_size)]

    def _selection(self, population):
        return random.choices(population, weights=[s.fitness for s in population], k=1)[0]

    def _crossover(self, parent_1, parent_2):
        bp = random.randrange(len(parent_1.code))
        problem = parent_1.problem
        child_1_code = parent_1.code[:bp] + parent_2.code[bp:]
        child_2_code = parent_2.code[:bp] + parent_1.code[bp:]
        return Solution(problem, child_1_code), Solution(problem, child_2_code)

    def _mutate(self, chromosome):
        if random.random() < self.mutation_probability:
            index = random.randrange(len(chromosome.code))
            chromosome.code[index] = not chromosome.code[index]
            chromosome.fitness = chromosome.calc_fitness()

    def solve(self, problem):
        el_bp = int(self.elitism * self.population_size // 2 * 2)
        population = sorted(self._generate_population(problem), reverse=True)
        new_population = population
        best_solution = population[0]
        gens_without_improvement = 0
        for i in range(self.num_of_generations):
            if gens_without_improvement == self.max_without_improvement:
                break
            new_population[0:el_bp] = population[0:el_bp]
            for j in range(el_bp, self.population_size, 2):
                parent_1 = self._selection(population)
                parent_2 = self._selection(population)
                child_1, child_2 = self._crossover(parent_1, parent_2)
                self._mutate(child_1)
                self._mutate(child_2)
                new_population[j] = child_1
                new_population[j + 1] = child_2
            population = sorted(new_population, reverse=True)
            best_in_current = population[0]
            if best_in_current > best_solution:
                best_solution = best_in_current
                gens_without_improvement = 0
            else:
                gens_without_improvement += 1
        return best_solution, i + 1 - gens_without_improvement

In [28]:
class LocalSearchOptimizer:
    def __init__(self, num_of_iterations=10):
        self.num_of_iterations = num_of_iterations

    def optimize(self, solution):
        current_solution = solution
        for _ in range(self.num_of_iterations):
            new_solution = current_solution.get_inverted()
            if new_solution > current_solution:
                current_solution = new_solution
        return current_solution

p = 1.0 / i ** c
![](../riProj/simAnnC.gif)

In [29]:
class SimulatedAnnealingOptimizer:
    def __init__(self, num_of_iterations=10, annealing_param=0.5):
        self.num_of_iteratiaons = num_of_iterations
        self.annealing_param = annealing_param  # <-(0,1) :  (>0.5 less chance | <0.5 more chance) ->  to accept worse solution

    def optimize(self, solution):
        current_solution = solution
        best_solution = solution
        # if we want to do annealing in first iter range should start from 1 (i=1 -> p=1, q=[0,1) -> p>q)
        for i in range (2, self.num_of_iteratiaons + 2):
            new_solution = current_solution.get_inverted()
            if new_solution > current_solution:
                current_solution = new_solution
                if new_solution > best_solution:
                    best_solution = new_solution
            else:
                p = 1 / i**self.annealing_param
                q = random.uniform(0, 1)
                if q < p:
                    current_solution = new_solution
        return best_solution

In [30]:
class RVNSOptimizer:
    def __init__(self, num_of_iterations=10, num_of_neighbours=3):
        self.num_of_iterations = num_of_iterations
        self.num_of_neighbours = num_of_neighbours

    def optimize(self, solution):
        current_solution = solution
        for _ in range(self.num_of_iterations):
            for k in range(1, self.num_of_neighbours + 1):
                new_solution = current_solution.get_inverted(k)
                if new_solution > current_solution:
                    current_solution = new_solution
                    break
        return current_solution

In [40]:
class MemeticSolver(GeneticSolver):
    def __init__(self, local_optimizer, opt_frequency=5, num_of_generations=50, max_without_improvement=20, population_size=10, mutation_probability=0.05, elitism=0.2):
        super().__init__(num_of_generations, max_without_improvement, population_size, mutation_probability, elitism)
        self.local_optimizer = local_optimizer
        self.opt_frequency = opt_frequency
        
    def solve(self, problem):
        el_bp = int(self.elitism * self.population_size // 2 * 2)
        population = sorted(self._generate_population(problem), reverse=True)
        new_population = population
        best_solution = population[0]
        gens_without_improvement = 0
        for i in range(self.num_of_generations):
            if gens_without_improvement == self.max_without_improvement:
                break
            new_population[0:el_bp] = population[0:el_bp]
            opt_counter = 0
            for j in range(el_bp, self.population_size, 2):
                parent_1 = self._selection(population)
                parent_2 = self._selection(population)
                child_1, child_2 = self._crossover(parent_1, parent_2)
                self._mutate(child_1)
                self._mutate(child_2)
                if opt_counter % self.opt_frequency == 0:
                    child_1 = self.local_optimizer.optimize(child_1)
                    child_2 = self.local_optimizer.optimize(child_2)
                opt_counter += 1
                new_population[j] = child_1
                new_population[j + 1] = child_2
            population = sorted(new_population, reverse=True)
            best_in_current = population[0]
            if best_in_current > best_solution:
                best_solution = best_in_current
                gens_without_improvement = 0
            else:
                gens_without_improvement += 1
        return best_solution, i + 1 - gens_without_improvement

In [41]:
class SolverExperiment:
    def __init__(self, problem, solver, num_of_runs):
        self.problem = problem
        self.solver = solver
        self.num_of_runs = num_of_runs
    
    def run_experiment(self):
        stats = {}
        stats['num_of_formulae'] = len(self.problem.cnf_list)
        stats['formula_num_of_clauses'] = self.problem.cnf_list[0].num_of_clauses
        stats['num_of_vars'] = self.problem.num_of_vars
        solutions = []
        times = []
        iters_till_best = []
        for _ in range(self.num_of_runs):
            start_time = time.time()
            solution, iters = self.solver.solve(self.problem)
            end_time = time.time()
            solutions.append(solution)
            times.append(end_time - start_time)
            iters_till_best.append(iters)
        stats['best_solution_sat'] = max(solutions).get_num_sat()
        stats['average_solution_sat'] = sum([solution.get_num_sat() for solution in solutions]) / len(solutions)
        stats['worst_solution_sat'] = min(solutions).get_num_sat()
        stats['average_running_time'] = sum(times) / len(times)
        stats['average_iters_till_best'] = sum(iters_till_best) / len(iters_till_best)
        return stats

In [39]:
instance_dir = 'problem_instances/'
instance_name = 'small1_50_7_1'
p = Problem(f'{instance_dir}{instance_name}')

bf_solver = BFSolver()
ga_solver = GeneticSolver(num_of_generations=100, population_size=20)
ma_ls_solver = MemeticSolver(LocalSearchOptimizer(), num_of_generations=100, population_size=20)
ma_sa_solver = MemeticSolver(SimulatedAnnealingOptimizer(), num_of_generations=100, population_size=20)
ma_rvns_solver = MemeticSolver(RVNSOptimizer(), num_of_generations=100, population_size=20)

experiment_bf = SolverExperiment(p, bf_solver, 100)
experiment_ga = SolverExperiment(p, ga_solver, 100)
experiment_ma_ls = SolverExperiment(p, ma_ls_solver, 100)
experiment_ma_sa = SolverExperiment(p, ma_sa_solver, 100)
experiment_ma_rvns = SolverExperiment(p, ma_rvns_solver, 100)

print(instance_name)
print(f'bf_solver: {experiment_bf.run_experiment()}')
print(f'ga_solver: {experiment_ga.run_experiment()}')
print(f'ma_ls_solver: {experiment_ma_ls.run_experiment()}')
print(f'ma_sa_solver: {experiment_ma_sa.run_experiment()}')
print(f'ma_rvns_solver: {experiment_ma_rvns.run_experiment()}')

small1_50_7_1
bf_solver: {'num_of_formulae': 50, 'formula_num_of_clauses': 1, 'num_of_vars': 7, 'best_solution_sat': 40, 'average_solution_sat': 40.0, 'worst_solution_sat': 40, 'average_running_time': 0.01440809965133667, 'average_iters_till_best': 128.0}
ga_solver: {'num_of_formulae': 50, 'formula_num_of_clauses': 1, 'num_of_vars': 7, 'best_solution_sat': 40, 'average_solution_sat': 40.0, 'worst_solution_sat': 40, 'average_running_time': 0.052728874683380125, 'average_iters_till_best': 1.74}
ma_ls_solver: {'num_of_formulae': 50, 'formula_num_of_clauses': 1, 'num_of_vars': 7, 'best_solution_sat': 40, 'average_solution_sat': 40.0, 'worst_solution_sat': 40, 'average_running_time': 0.16770053863525392, 'average_iters_till_best': 1.24}
ma_sa_solver: {'num_of_formulae': 50, 'formula_num_of_clauses': 1, 'num_of_vars': 7, 'best_solution_sat': 40, 'average_solution_sat': 40.0, 'worst_solution_sat': 40, 'average_running_time': 0.16807809591293335, 'average_iters_till_best': 1.34}
ma_rvns_solver